In [1]:
import pandas as pd
from Bio import SeqIO, SeqFeature
import os
from collections import OrderedDict
from pprint import pprint

In [2]:
meta = pd.read_csv('./hcv-all/prepare-reference-tree/data/hcv-meta.tsv', sep='\t', index_col='strain')
nextstrain_clade_meta = pd.read_csv('./giant_tree_output_meta.tsv', sep='\t', index_col='strain', dtype={'clade_membership': str})
seqs = [s for s in SeqIO.parse(open('./hcv-all/prepare-reference-tree/data/hcv-sequences.fasta', 'r'), 'fasta')]

In [8]:
meta['clade_membership'] = nextstrain_clade_meta['clade_membership']

In [6]:
for i in range(1, 9):
    genotype = "%d"%i
    
    gt_meta = meta.loc[meta['clade_membership'] == genotype]
    gt_seqs = [s for s in seqs if s.id in gt_meta.index.values]
       
    ictv_gt_meta = gt_meta.loc[gt_meta['source'] == 'ictv']
    
    paths = ['./hcv-%d'%i, './hcv-%d/prepare-reference-tree/'%i, './hcv-%d/prepare-reference-tree/data/'%i]

    for path in paths: 
        try:
            os.mkdir(path)
        except:
            continue

    metasuffix = 'hcv-%d-meta.tsv'%i
    seqsuffix = 'hcv-%d-sequences.fasta'%i

        
    gt_meta.to_csv(paths[-1]+metasuffix, sep='\t')
    SeqIO.write(gt_seqs, open(paths[-1]+seqsuffix, 'w'), 'fasta')
 
    f = open(paths[-1] + 'include.tsv', 'w')
    for strain in ictv_gt_meta.index.values:
        f.write(strain + '\n')
    f.close()

# Format reference sequences in genbank and gff formats, relabeling mat_peptides as genes


In [2]:

def makeNewFeatures(refseq): 
    
    newFeatures = []
    for f in refseq.features:
        if f.type in ["5'UTR", "3'UTR"]:
            newFeatures.append(f)
        elif f.type == 'mat_peptide':

            attrs = {}
            attrs['location'] = f.location

            product = f.qualifiers['product'][0]
            if product == 'core protein':
                attrs['id'] = 'C'
            else:
                attrs['id'] = product.split()[0]

            attrs['qualifiers'] = OrderedDict()
            attrs['qualifiers']['gene'] = [attrs['id']]

            newFeatures.append(SeqFeature.SeqFeature(type='gene', **attrs))
            newFeatures.append(SeqFeature.SeqFeature(type='CDS', **attrs))


        elif 'product' in f.qualifiers.keys() and f.qualifiers['product'][0] == 'protein F':

            attrs = {}
            attrs['qualifiers'] = OrderedDict()

            attrs['id'] = 'F-part1'
            attrs['location'] = f.location.parts[0]
            attrs['qualifiers']['gene'] = ['F-part1']
            newFeatures.append(SeqFeature.SeqFeature(type='gene', **attrs))
            newFeatures.append(SeqFeature.SeqFeature(type='CDS', **attrs))

            attrs2 = {}
            attrs2['qualifiers'] = OrderedDict()
            attrs2['id'] = 'F-part2'
            attrs2['location'] = f.location.parts[1]
            attrs2['qualifiers']['gene'] = ['F-part2']
            attrs2['qualifiers']['codon_start'] = [1]
            newFeatures.append(SeqFeature.SeqFeature(type='gene', **attrs2))
            newFeatures.append(SeqFeature.SeqFeature(type='CDS', **attrs2))

    refseq.features = newFeatures
    return refseq

In [3]:
def make_gff(refseq, path): 
    def feature_to_dict(seqfeature, refseq=refseq): 
        return {
            'seqname': refseq.annotations['accessions'][0],
            'source': 'feature',
            'feature': 'gene',
            'start': int(seqfeature.location.start),
            'end': int(seqfeature.location.end),
            'score': '.',
            'strand': '+',
            'frame': '1' if seqfeature.id == 'F-part2' else 0,
            'attribute': 'gene='+seqfeature.id
        }
    
    header_line0 = '## sequence-region\t' + refseq.annotations['accessions'][0] + '\t' + '0' + '\t' + str(len(refseq)+1)
    header_line1 = '## seqname	source	feature	start	end	score	strand	frame	attribute'
    gff_lines = [feature_to_dict(feature, refseq) for feature in refseq.features if feature.type == 'gene']
    gff_df = pd.DataFrame(gff_lines, columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
    
    with open(path, 'a') as f:
        f.write(header_line0+'\n')
        f.write(header_line1+'\n')
        gff_df.to_csv(f, header=False, index=False, sep='\t')

In [4]:
# reference_genomes = {'hcv-%d'%i : SeqIO.read(open('./more-reference-genomes/hcv-%d-reference.gb'%i, 'r'), 'genbank') for i in range(3, 8)}

reference_genomes = {'hcv-2': SeqIO.read(open('../hcv-2/reference-files/hcv-2-reference.gb', 'r'), 'genbank')}

for genotype, refseq in reference_genomes.items():
    
    paths = ['../%s/'%genotype, '../%s/reference-files/'%genotype]
    
    fasta_fname = '%s-reference.fasta'%genotype
    gb_fname = '%s-reference.gb'%genotype
    gff_fname = '%s-genemap.gff'%genotype

    for path in paths: 
        try:
            os.mkdir(path)
        except:
            continue
    
    new_ref_seq = makeNewFeatures(refseq)
    new_ref_seq.id = new_ref_seq.annotations['accessions'][0]
    new_ref_seq.description = new_ref_seq.annotations['accessions'][0]
    reference_genomes[genotype] = new_ref_seq
    SeqIO.write(new_ref_seq, open(paths[-1]+fasta_fname, 'w'), 'fasta')
    SeqIO.write(new_ref_seq, open(paths[-1]+gb_fname, 'w'), 'genbank')
    make_gff(new_ref_seq, paths[-1]+gff_fname)
    